# E-mail Spam Filtering Extra Credit

In [1]:
# coding: utf-8

#Simple NB Based Lingspam Spam classifier 
import numpy as np
import sklearn
import sklearn.datasets as skd
from scipy.sparse import csc_matrix
from sklearn.feature_extraction.text import CountVectorizer
from sklearn.metrics import mutual_info_score

from sklearn import linear_model
from sklearn import naive_bayes

#Each sub-directory in the parent directory is assumed to contain documents from the same class
#I pre-processed the part1 (fold1) and part2 (fold2) of the lingspam dataset to place spam emails in one folder 
#and legit emails in another; you should do the same for the entire dataset, either manually or via a script. 
ls_train = skd.load_files('./data/lingspam_public/lemm_stop/train');
ls_test = skd.load_files('./data/lingspam_public/lemm_stop/test');

#The count vectorizer classes fit_transform function generates a vocoabulary that contains each unique term in the dataset
#and outputs a sparse matrix tabulating term occurences
count_vect = CountVectorizer(binary=True)
x_train = count_vect.fit_transform(ls_train.data)

#Since the vocabulary has already been learned, use the transform function to transform the test data using the same vocab
x_test = count_vect.transform(ls_test.data)

In [2]:
# Prepare
num_email = x_train.shape[0]
num_feature = x_train.shape[1]

# Transport saved data from sparse matrix out
x_train_data = x_train.toarray()
x_train_ig = np.zeros([num_feature])

for i in range(num_feature):
    
    # Each colunm show the occurence of one feature in all emails
    feature_vector = x_train_data[:,i]
    
    # Reshape 
    feature_vector = feature_vector.reshape([num_email])
    
    # Calculate ig for features 
    x_train_ig [i] = mutual_info_score(feature_vector, ls_train.target)

# Use sorting function to sort the ig array, add minus '-' for descending order 
x_train_ig_sort = np.argsort(-x_train_ig)

# Extract feature names
name_feature = count_vect.get_feature_names()

# Select N largest features' index
N = 10
top_feature = np.array(x_train_ig_sort[:N])
drop_feature = np.array(x_train_ig_sort[N:num_feature])

In [3]:
def binary_feature(ls_train, ls_test):
    # Use the count vectorizer classes to get binary featrues
    # Set parmeter 'binary' to True, all non zero counts are set to 1
    count_vect_bf = CountVectorizer(binary=True)
    
    x_train_bf = count_vect_bf.fit_transform(ls_train.data)
    x_test_bf  = count_vect_bf.transform(ls_test.data)
    
    # Still drop the unwanted features in training set
    x_train_bf = x_train_bf[:, top_feature]
    x_test_bf  = x_test_bf[:, top_feature]
    
    return x_train_bf, x_test_bf

x_train_bf, x_test_bf = binary_feature(ls_train, ls_test)

In [4]:
from sklearn.metrics import precision_recall_fscore_support 

def pre_rec(yts, yhat):
    precision, recall, f1,_ = precision_recall_fscore_support(yts,
                                                              yhat,
                                                              average='binary')
    return precision, recall

In [5]:
# Multinomial NB with BF features
mNomBF = sklearn.naive_bayes.MultinomialNB()
mNomBF.fit(x_train_bf,ls_train.target)

y_predict_M_BF = mNomBF.predict(x_test_bf)

pre,rec = pre_rec(ls_test.target, y_predict_M_BF)

print('Original Classifier')
print('Precision = {0:f}'.format(pre))
print('Recall= {0:f}'.format(rec))

Original Classifier
Precision = 0.888889
Recall= 0.816327


Calculate the log_odds.

In [6]:
num_x_spam = 0
num_x_legit = 0
num_spam_add = 0
num_legit_add = 0

for i in range(N):
    num_spam_add = np.count_nonzero((x_test_bf.toarray()[:,i])&(y_predict_M_BF==1))
    num_legit_add = np.count_nonzero((x_test_bf.toarray()[:,i])&(y_predict_M_BF==0))
    
    num_x_spam = num_x_spam + num_spam_add
    num_x_legit = num_x_legit + num_legit_add

def log_odds(i, x, y_predict):
    
    num_spam  = np.count_nonzero(y_predict==1)
    num_legit = np.count_nonzero(y_predict==0)
    
    num_xi_spam  = np.count_nonzero(x[:,i]&(y_predict_M_BF==1))
    num_xi_legit = np.count_nonzero(x[:,i]&(y_predict_M_BF==0))
    
    log_odd = np.log((num_xi_spam+1/num_x_spam+N)/(num_xi_legit+1/num_x_legit+N))
    
    return log_odd

Find featrues with contribution to legit probability.

In [7]:
x_test_init = count_vect.transform(ls_test.data)[:, top_feature]
x_test_temp = count_vect.transform(ls_test.data)[:, top_feature]

log_delta = 0
cost = []
edit_feature = []
ind_spam = np.array(ls_test.target==1)
num_spam = np.count_nonzero(ls_test.target==1)

for i,feature_ind in enumerate(top_feature):
    
    log_1 = log_odds(i,x_test_bf.toarray(),y_predict_M_BF)
    
    x_test_temp[:,i] = 1
    y_predict_temp = mNomBF.predict(x_test_temp)
    log_2 = log_odds(i,x_test_temp.toarray(),y_predict_temp)
    
    delta_log = log_1 - log_2
    costi = np.count_nonzero(x_test_init.toarray()[ind_spam,i] == 0)

    if (((delta_log)<=0)):
        cost.append(costi)
        edit_feature.append(i)
        
        print('The ' + str(i) + 'th feature could help the attackers.')
        print('It has ' + str(costi) + ' unit cost in test set.')

The 0th feature could help the attackers.
It has 49 unit cost in test set.
The 3th feature could help the attackers.
It has 49 unit cost in test set.
The 4th feature could help the attackers.
It has 48 unit cost in test set.


C:\Users\xiaos\Anaconda3\lib\site-packages\scipy\sparse\compressed.py:774: SparseEfficiencyWarning: Changing the sparsity structure of a csr_matrix is expensive. lil_matrix is more efficient.
  SparseEfficiencyWarning)


Use Gray Code to generate possible modified vectors.

In [8]:
from sympy.combinatorics.graycode import GrayCode

a = GrayCode(3)
y = list(a.generate_gray())

z = np.zeros([8,3])
for i in range(8):
    z[i,:] = list(y[i])

z.astype(int)

array([[0, 0, 0],
       [0, 0, 1],
       [0, 1, 1],
       [0, 1, 0],
       [1, 1, 0],
       [1, 1, 1],
       [1, 0, 1],
       [1, 0, 0]])

In [9]:
cost_all = np.sum(cost)

for i in range(len(z)):
    
    x_test_new_i = count_vect.transform(ls_test.data)[:, top_feature]
    
    x_test_new_i[ind_spam,0] = z[i,0]
    x_test_new_i[ind_spam,3] = z[i,1]
    x_test_new_i[ind_spam,4] = z[i,2]
    
    
    y_predict = mNomBF.predict(x_test_new_i)
    
    pre, rec = pre_rec(ls_test.target,y_predict)
    cost_edit = np.sum(np.dot(z[i,0], cost))
    
    if (pre==0):
        if cost_edit <= cost_all:
            print(z[i])
            print('Precision is ' + str(pre))
            cost_all = cost_edit
    
    if ((i==8)&(pre!=0)):
        print('No answer')

C:\Users\xiaos\Anaconda3\lib\site-packages\scipy\sparse\compressed.py:774: SparseEfficiencyWarning: Changing the sparsity structure of a csr_matrix is expensive. lil_matrix is more efficient.
  SparseEfficiencyWarning)


[ 1.  1.  1.]
Precision is 0.0


Modified the test set and predict it with original classifier.

In [10]:
x_test_new_attack = count_vect.transform(ls_test.data)[:, top_feature]
x_test_new_attack[ind_spam,0] = 1
x_test_new_attack[ind_spam,3] = 1
x_test_new_attack[ind_spam,4] = 1

y_predict_attack = mNomBF.predict(x_test_new_attack)

fn = np.count_nonzero((ls_test.target==1) & (y_predict_M_BF==0))/num_spam
fn_attack = np.count_nonzero((ls_test.target==1) & (y_predict_attack==0))/num_spam

print('False Negative Rate Before Modification = {0:f}'.format(fn))
print('False Negative Rate After  Modification = {0:f}'.format(fn_attack))
print('Minimum unit cost to fool the spam filter is '+ str(cost_edit/num_spam))

False Negative Rate Before Modification = 0.183673
False Negative Rate After  Modification = 1.000000
Minimum unit cost to fool the spam filter is 2.97959183673


C:\Users\xiaos\Anaconda3\lib\site-packages\scipy\sparse\compressed.py:774: SparseEfficiencyWarning: Changing the sparsity structure of a csr_matrix is expensive. lil_matrix is more efficient.
  SparseEfficiencyWarning)


## Filter Update
Adding new edited spam emails to the original training set.

In [11]:
from sklearn.model_selection import KFold

spam_email = x_test_bf.toarray()[ind_spam]
spam_email_target = ls_test.target[ind_spam]

x_train_bf_new = count_vect.transform(ls_train.data)[:, top_feature]
x_train_bf_new = np.vstack((x_train_bf.toarray(),spam_email))

ls_train_new = np.hstack((ls_train.target,spam_email_target))

Train the original filter agian.

In [12]:
fn_update = []
fp_update = []
acc_update = []

nfold = 10
kf = KFold(n_splits=nfold,shuffle=True)

for train, test in kf.split(x_train_bf_new):
    Xtr = x_train_bf_new[train,:]
    Xts = x_train_bf_new[test,:]
    ytr = ls_train_new[train]
    yts = ls_train_new[test]

    mNomBF.fit(Xtr,ytr)

    yhat = mNomBF.predict(Xts)
    
    fn_update.append(np.mean((yts==1) & (yhat==0)))
    fp_update.append(np.mean((yts==0) & (yhat==1)))
    acc_update = np.mean(np.mean((yts==yhat)))

fn_update = np.mean(fn_update)
fp_update = np.mean(fp_update)
acc_update = np.mean(acc_update)

print('False Negative After Update = ' + str(fn_update))
print('False Positive After Update = ' + str(fp_update))
print('Classifier Accuracy = ' + str(acc_update)) 

False Negative After Update = 0.0248914739679
False Positive After Update = 0.0101858419634
Classifier Accuracy = 0.966037735849
